In [1]:
import pandas as pd
import json
from pandas import json_normalize

df = pd.read_csv('final_data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147519 entries, 0 to 147518
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   pos          147519 non-null  int64 
 1   artist_name  147519 non-null  object
 2   track_uri    147519 non-null  object
 3   artist_uri   147519 non-null  object
 4   track_name   147518 non-null  object
 5   album_uri    147519 non-null  object
 6   duration_ms  147519 non-null  int64 
 7   album_name   147519 non-null  object
 8   Genre        147519 non-null  object
dtypes: int64(2), object(7)
memory usage: 10.1+ MB


In [13]:
with open ('vetting_playlist.json', 'r') as file:
    vet = json.load(file)

<p>Dancability<br>
Year<br>
Genre<br>
Duration<br>
Popularity<br>
Liveliness<br>
Marked Explicit?</p>

In [8]:
genre = df['Genre']
genre.head(10)

0                      hip hop
1                          pop
2                       reggae
3                          r&b
4                          r&b
5    southern hip hop, hip hop
6                          r&b
7                          r&b
8               pop punk, punk
9                          r&b
Name: Genre, dtype: object

In [9]:
duration = df['duration_ms']
duration.head(10)

0    226863
1    198800
2    227600
3    250373
4    223440
5    235213
6    256426
7    229866
8    193042
9    229040
Name: duration_ms, dtype: int64

In [10]:
dfm = pd.merge('Genre', 'duration_ms')

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed